In [1]:
%load_ext memory_profiler
import pandas as pd
import random
import time

In [2]:
def gen_data(num_of_raw: int=10000, num_of_ele: int=10) -> pd.DataFrame:
    lst_ele = [chr(i) for i in range(97, 97+num_of_ele)]
    row1 = random.choices(lst_ele, k=num_of_raw)
    row2 = random.choices(lst_ele, k=num_of_raw)
    row3 = random.choices(lst_ele, k=num_of_raw)
    row4 = random.choices(lst_ele, k=num_of_raw)
    row5 = random.choices(lst_ele, k=num_of_raw)
    row6 = random.choices(lst_ele, k=num_of_raw)
    row7 = random.choices(lst_ele, k=num_of_raw)
    row8 = random.choices(lst_ele, k=num_of_raw)
    row9 = random.choices(lst_ele, k=num_of_raw)
    data = {
        "Tool": row1,
        "Prev1Tool": row2,
        "Prev2Tool": row3,
        "other1": row4,
        "other2": row5,
        "other3": row6,
        "other4": row7,
        "other5": row8,
        "other6": row9,
        }
    df_res = pd.DataFrame(data=data)
    return df_res

def foo1(df):
    if (df["Tool"] == df["Prev1Tool"]) and (df["Prev1Tool"] == df["Prev2Tool"]):
        df["Trans1"] = "A"
        df["Trans2"] = "A"
        df["Trans3"] = "A"
    elif (df["Tool"] == df["Prev1Tool"]) and (df["Prev1Tool"] != df["Prev2Tool"]):
        df["Trans1"] = "A"
        df["Trans2"] = "A"
        df["Trans3"] = "B"
    elif (df["Tool"] != df["Prev1Tool"]) and (df["Prev1Tool"] == df["Prev2Tool"]):
        df["Trans1"] = "B"
        df["Trans2"] = "A"
        df["Trans3"] = "A"
    elif (df["Tool"] != df["Prev1Tool"]) and (df["Tool"] == df["Prev2Tool"]):
        df["Trans1"] = "A"
        df["Trans2"] = "B"
        df["Trans3"] = "A"
    else:
        df["Trans1"] = "A"
        df["Trans2"] = "B"
        df["Trans3"] = "C"
    return df

def foo2(df):
    if (df["Tool"] == df["Prev1Tool"]) and (df["Prev1Tool"] == df["Prev2Tool"]):
        return "A","A","A"
    elif (df["Tool"] == df["Prev1Tool"]) and (df["Prev1Tool"] != df["Prev2Tool"]):
        return "A","A","B"
    elif (df["Tool"] != df["Prev1Tool"]) and (df["Prev1Tool"] == df["Prev2Tool"]):
        return "B","A","A"
    elif (df["Tool"] != df["Prev1Tool"]) and (df["Tool"] == df["Prev2Tool"]):
        return "A","B","A"
    else:
        return "A","B","C"

def method_1(df_rawdata):
    df_data = df_rawdata.copy()
    df_data = df_data.apply(foo1, axis=1)
    return df_data

def method_2(df_rawdata):
    df_data = df_rawdata.copy()
    df_data["tran1"], df_data["trans2"], df_data["trans3"] = zip(*df_data.apply(foo2, axis=1))
    return df_data

In [3]:
LST_TOOL = ["Tool", "Prev1Tool", "Prev2Tool"]
NUM_OF_ROW = 100000
NUM_OF_ELE = 100

df_rawdata = gen_data(num_of_raw=NUM_OF_ROW, num_of_ele=NUM_OF_ELE)
print(df_rawdata.head())

  Tool Prev1Tool Prev2Tool other1 other2 other3 other4 other5 other6
0             Â         £      b      ¦      Ä            a      µ
1    {         i         |      t      {      À                  }
2    °         b         ¶      £      q      ©                  »
3    ¾         ©         ©      ­      ¸      ±      §            
4    ¤         °         l            »                  ³      


In [4]:
print("===== Method 1 =====")
%timeit -r 5 -n 10 df_res = method_1(df_rawdata)

print("===== Method 2 =====")
%timeit -r 5 -n 10 df_res = method_2(df_rawdata)

===== Method 1 =====
44 s ± 61.1 ms per loop (mean ± std. dev. of 5 runs, 10 loops each)
===== Method 2 =====
1.12 s ± 3.83 ms per loop (mean ± std. dev. of 5 runs, 10 loops each)


In [5]:
print("===== Method 1 =====")
%memit method_1(df_rawdata)

===== Method 1 =====
peak memory: 423.73 MiB, increment: 218.64 MiB


In [6]:
print("===== Method 2 =====")
%memit method_2(df_rawdata)

===== Method 2 =====
peak memory: 223.45 MiB, increment: 12.98 MiB


In [7]:
lst_m1_time = []
lst_m2_time = []
lst_test_num = [10**n for n in range(0,8)]
for num_of_row in lst_test_num:
    _df_rawdata = gen_data(num_of_raw=num_of_row, num_of_ele=NUM_OF_ELE)
    t1 = time.perf_counter()
    _ = method_1(_df_rawdata)
    t2 = time.perf_counter()
    _ = method_2(_df_rawdata)
    t3 = time.perf_counter()
    
    lst_m1_time.append(t2-t1)
    lst_m2_time.append(t3-t2)

In [8]:
df_res = pd.DataFrame({"Method_Old": lst_m1_time, "Method_New": lst_m2_time})
df_res.to_csv("./df_compare.csv")